In [1]:
from barlowbert_dm import DataCollatorForBarlowBertWithMLM
from barlowbert_models import BarlowBert
from transformers import BertTokenizerFast

from datasets import load_dataset, load_from_disk, concatenate_datasets
import torch
from tqdm import tqdm

In [2]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', padding=True,truncation=True,)
collator = DataCollatorForBarlowBertWithMLM(tokenizer,mlm_probability=0.0)

In [3]:
tokenizer.encode_plus('i am haris',truncation=True,padding='max_length',max_length=32,return_tensors='pt')

{'input_ids': tensor([[  101,  1045,  2572, 21291,  2015,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}

In [4]:
corpus = load_dataset('bookcorpus',split='train')

Reusing dataset bookcorpus (/mounts/data/corp/huggingface/datasets/bookcorpus/plain_text/1.0.0/44662c4a114441c35200992bea923b170e6f13f2f0beb7c14e43759cec498700)


In [5]:
mini = corpus.select(range(1000))

In [6]:
dataset = mini.map(lambda x: tokenizer.encode_plus(x['text'],return_token_type_ids=False,padding='max_length',max_length=128,truncation=True),remove_columns=['text'],num_proc=20)

In [7]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [8]:
# dataset.save_to_disk("/mounts/data/proj/jabbar/barlowbert/bookcorpus_all_128")

In [11]:
# data = load_from_disk('/mounts/unts/data/proj/jabbar/barlowbert/bookcorpus_20mil_128')
# data.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [13]:
for i in tqdm(range(len(dataset))):
    if dataset[i]['input_ids'].shape[0]!=128:
        print(i)

100%|██████████| 1000/1000 [00:01<00:00, 732.85it/s]


In [14]:
dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  1996,  2431,  1011, 17002,  2338,  2028,  1999,  1996,  2991,
          1997, 16270, 11510,  2401,  2186, 10905, 10559,  2061,  4063,  4645,
          9385,  2286, 10905, 10559,  2061,  4063,  4645,  2035,  2916,  9235,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     

In [15]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=4, num_workers=2,
                                            pin_memory=True, collate_fn=collator)

In [16]:
collator

DataCollatorForBarlowBertWithMLM(tokenizer=PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.0, pad_to_multiple_of=None)

In [ ]:
batch = next(iter(train_loader))

In [ ]:
(x,y) = batch

In [ ]:
x['input_ids'][0]

In [ ]:
y['input_ids'][0]

In [ ]:
x['input_ids'][0]==y['input_ids'][0]